In [1]:
#r "nuget: Apache.Ignite, 2.15.0"
#r "nuget: Apache.Ignite.Linq, 2.15.0"

Installed Packages Apache.Ignite, 2.15.0 Apache.Ignite.Linq, 2.15.0

In [2]:
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Threading.Tasks;
using Apache.Ignite.Core;
using Apache.Ignite.Core.Cache;
using Apache.Ignite.Core.Cache.Configuration;

In [3]:
class Country
{
    public int Id { get; set; }
    public string Name { get; set; }
    public string Iso3 { get; set; }
    public string Iso2 { get; set; }
    public string NumericCode { get; set; }
    public string PhoneCode { get; set; }
    public string Capital { get; set; }
    public string Currency { get; set; }
    public string CurrencyName { get; set; }
    public string CurrencySymbol { get; set; }
    public string Tld { get; set; }
    public string Native { get; set; }
    public string Region { get; set; }
    public string RegionId { get; set; }
    public string Subregion { get; set; }
    public string SubregionId { get; set; }
    public string Nationality { get; set; }
    public List<TimeZone> Timezones { get; set; }
    public Translations Translations { get; set; }
    public string Latitude { get; set; }
    public string Longitude { get; set; }
    public string Emoji { get; set; }
    public string EmojiU { get; set; }
}

class TimeZone
{
    public string ZoneName { get; set; }
    public int GmtOffset { get; set; }
    public string GmtOffsetName { get; set; }
    public string Abbreviation { get; set; }
    public string TzName { get; set; }
}

class Translations
{
    public string Kr { get; set; }
    public string PtBr { get; set; }
    public string Pt { get; set; }
    public string Nl { get; set; }
    public string Hr { get; set; }
    public string Fa { get; set; }
    public string De { get; set; }
    public string Es { get; set; }
    public string Fr { get; set; }
    public string Ja { get; set; }
    public string It { get; set; }
    public string Cn { get; set; }
    public string Tr { get; set; }
}

In [4]:
async Task<string> FetchJsonDataAsync(string url)
{
    using (var httpClient = new HttpClient())
    {
        return await httpClient.GetStringAsync(url);
    }
}

var countriesJsonUrl = "https://raw.githubusercontent.com/dr5hn/countries-states-cities-database/master/countries.json";
var countriesJson = await FetchJsonDataAsync(countriesJsonUrl);

In [5]:
var igniteConfig = new IgniteConfiguration
{
    CacheConfiguration = new[]
    {
        new CacheConfiguration("countriesCache")
        {
            QueryEntities = new[]
            {
                new QueryEntity
                {
                    KeyType = typeof(int),
                    ValueType = typeof(Country),
                    Fields = new[]
                    {
                        new QueryField("Id", typeof(int)),
                        new QueryField("Name", typeof(string)),
                        new QueryField("Region", typeof(string)),
                        new QueryField("Subregion", typeof(string)),
                        new QueryField("Capital", typeof(string)),
                        new QueryField("Currency", typeof(string)),
                        // Add other fields for indexing
                    }
                }
            },
            SqlSchema = "PUBLIC", // Change the schema if needed
            Backups = 1, // Number of backups for data redundancy
            WriteSynchronizationMode = CacheWriteSynchronizationMode.FullSync,
            AtomicityMode = CacheAtomicityMode.Transactional,
            CacheMode = CacheMode.Partitioned, // Change to CacheMode.Replicated if needed
            OnheapCacheEnabled = true,
            QueryParallelism = 2, // Adjust based on the available CPU cores
            // Add more cache configuration options as needed
        }
    },
    // Add more Ignite configuration options as needed
};


In [59]:

using (var ignite = Ignition.Start(igniteConfig))
{
    // Create a cache
    var cache = ignite.GetOrCreateCache<int, Country>("countriesCache");

    // Deserialize JSON data and insert into the cache
    var countries = Newtonsoft.Json.JsonConvert.DeserializeObject<List<Country>>(countriesJson);

    foreach (var country in countries)
    {
        cache.Put(country.Id, country);
    }

    // Display the number of countries in the cache
    Console.WriteLine($"Number of countries in cache: {cache.GetSize()}");

    // Sample query
    var query = cache.Query(new Apache.Ignite.Core.Cache.Query.SqlFieldsQuery("SELECT Name FROM Country WHERE Region = 'Asia'"));

    foreach (var result in query)
    {
        Console.WriteLine($"Country in Asia: {result[0]}");
    }

    // Sample transaction
    using (var tx = ignite.GetTransactions().TxStart())
    {
        // Perform transactional operations
        var countryToUpdate = cache.Get(1);
        countryToUpdate.Name = "Updated Country";
        cache.Put(1, countryToUpdate);
        Console.WriteLine("Sample transaction");
        // Commit the transaction
        tx.Commit();
    }

    // Sample concurrency control (optimistic)
    var updatedCountry = new Country { Id = 1, Name = "Updated Value" };

    // Use GetAndPutIfAbsent as a workaround for optimistic concurrency control
    var existingCountry = cache.GetAndPutIfAbsent(1, updatedCountry);

    if (existingCountry == null)
    {
        Console.WriteLine("Successfully updated value");
    }
    else
    {
        Console.WriteLine("Failed to update due to concurrency control");
    }

    var newQ = cache.Query(new Apache.Ignite.Core.Cache.Query.SqlFieldsQuery("SELECT Name FROM Country WHERE Id = 1"));

    foreach (var result in newQ)
    {
        Console.WriteLine($"Country in Asia: {result[0]}");
    }
    Ignition.Stop(ignite.Name, true);
}



[21:47:22,439][WARNING][main][G] Ignite work directory is not provided, automatically resolved to: /Users/kiettran/Projects/weather-db/analysis/ignite/work
[21:47:22,440][INFO][main][IgniteKernal] 

>>>    __________  ________________  
>>>   /  _/ ___/ |/ /  _/_  __/ __/  
>>>  _/ // (7 7    // /  / / / _/    
>>> /___/\___/_/|_/___/ /_/ /___/   
>>> 
>>> ver. 2.15.0#20230425-sha1:f98f7f35
>>> 2023 Copyright(C) Apache Software Foundation
>>> 
>>> Ignite documentation: https://ignite.apache.org

[21:47:22,441][INFO][main][IgniteKernal] Config URL: n/a
[21:47:22,441][INFO][main][IgniteKernal] IgniteConfiguration [igniteInstanceName=null, pubPoolSize=8, svcPoolSize=8, callbackPoolSize=8, stripedPoolSize=8, sysPoolSize=8, mgmtPoolSize=4, dataStreamerPoolSize=8, utilityCachePoolSize=8, utilityCacheKeepAliveTime=60000, p2pPoolSize=2, qryPoolSize=8, buildIdxPoolSize=2, igniteHome=null, igniteWorkDir=/Users/kiettran/Projects/weather-db/analysis/ignite/work, mbeanSrv=com.sun.jmx.mbeanserver.Jm

Number of countries in cache: 250
Country in Asia: Armenia
Country in Asia: Azerbaijan
Country in Asia: Bahrain
Country in Asia: Bhutan
Country in Asia: Hong Kong S.A.R.
Country in Asia: Indonesia
Country in Asia: Iraq
Country in Asia: Israel
Country in Asia: Kazakhstan
Country in Asia: South Korea
Country in Asia: Kyrgyzstan
Country in Asia: Macau S.A.R.
Country in Asia: Malaysia
Country in Asia: Mongolia
Country in Asia: Nepal
Country in Asia: Oman
Country in Asia: Philippines
Country in Asia: Saudi Arabia
Country in Asia: Sri Lanka
Country in Asia: Taiwan
Country in Asia: Turkmenistan
Country in Asia: Uzbekistan
Country in Asia: Vietnam
Country in Asia: Afghanistan
Country in Asia: Bangladesh
Country in Asia: Brunei
Country in Asia: Cambodia
Country in Asia: China
Country in Asia: East Timor
Country in Asia: Georgia
Country in Asia: India
Country in Asia: Iran
Country in Asia: Japan
Country in Asia: Jordan
Country in Asia: North Korea
Country in Asia: Kuwait
Country in Asia: Laos
Co

[21:47:23,934][INFO][main][GridTcpRestProtocol] Command protocol successfully stopped: TCP binary
[21:47:23,946][INFO][main][GridCacheProcessor] Stopped cache [cacheName=countriesCache]
[21:47:23,947][INFO][main][GridCacheProcessor] Stopped cache [cacheName=ignite-sys-cache]
[21:47:23,954][INFO][main][IgniteKernal] 

>>> +----------------------------------------------------------------------------------+
>>> Ignite ver. 2.15.0#20230425-sha1:f98f7f35de6dc76a9b69299154afaa2139a5ec6d stopped OK
>>> +----------------------------------------------------------------------------------+
>>> Grid uptime: 00:00:00.110




# SQLite

In [45]:
#r "nuget: Microsoft.Data.Sqlite, 8.0.0"


Installed Packages Microsoft.Data.Sqlite, 8.0.0

In [46]:
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Threading.Tasks;
using Microsoft.Data.Sqlite;

In [47]:
string databasePath = "countries.db";
string connectionString = $"Data Source={databasePath};";


In [48]:

using (var connection = new SqliteConnection(connectionString))
{
    connection.Open();
    string deletion = "DROP TABLE IF EXISTS Countries;"; // Add more tables as needed

    using (SqliteCommand createTableCommand = new SqliteCommand(deletion, connection))
    {
        createTableCommand.ExecuteNonQuery();
    }

    // Alternatively, you can drop and recreate the tables
    // using (var command = new SQLiteCommand(connection))
    // {
    //     command.CommandText = "DROP TABLE IF EXISTS TableName1; DROP TABLE IF EXISTS TableName2;"; // Add more tables as needed
    //     command.ExecuteNonQuery();
    //     // Recreate tables if needed
    //     command.CommandText = "CREATE TABLE TableName1 (...); CREATE TABLE TableName2 (...);"; // Add more tables as needed
    //     command.ExecuteNonQuery();
    // }
}




In [49]:

SqliteConnection connection = new SqliteConnection(connectionString);

// Create a table to store country data
string createTableQuery = @"
    CREATE TABLE IF NOT EXISTS Countries (
        Id INTEGER PRIMARY KEY,
        Name TEXT,
        Capital TEXT,
        Region TEXT,
        Subregion TEXT
    );
";
connection.Open();
using (SqliteCommand createTableCommand = new SqliteCommand(createTableQuery, connection))
{
    createTableCommand.ExecuteNonQuery();
}

// Indexing structures
string createIndexQuery = "CREATE INDEX IF NOT EXISTS idx_region ON Countries(Region);";
using (SqliteCommand createIndexCommand = new SqliteCommand(createIndexQuery, connection))
{
    createIndexCommand.ExecuteNonQuery();
}


// Deserialize JSON data into a list of Country objects
List<Country> countries = Newtonsoft.Json.JsonConvert.DeserializeObject<List<Country>>(countriesJson);

// Insert country data into the SQLite database
string insertDataQuery = "INSERT INTO Countries (Id, Name, Capital, Region, Subregion) VALUES (@Id, @Name, @Capital, @Region, @Subregion);";
using (SqliteTransaction transaction = connection.BeginTransaction())
{
    foreach (Country country in countries)
    {
        using (SqliteCommand insertCommand = new SqliteCommand(insertDataQuery, connection, transaction))
        {
            insertCommand.Parameters.AddWithValue("@Id", country.Id);
            insertCommand.Parameters.AddWithValue("@Name", country.Name);
            insertCommand.Parameters.AddWithValue("@Capital", country.Capital);
            insertCommand.Parameters.AddWithValue("@Region", country.Region);
            insertCommand.Parameters.AddWithValue("@Subregion", country.Subregion);
            insertCommand.ExecuteNonQuery();
        }
    }

    transaction.Commit();
}

// Query and print some data from the database
string selectQuery = "SELECT * FROM Countries WHERE Region = 'Asia' LIMIT 5;";
using (SqliteCommand selectCommand = new SqliteCommand(selectQuery, connection))
{
    using (SqliteDataReader reader = selectCommand.ExecuteReader())
    {
        Console.WriteLine("Query Results:");
        while (reader.Read())
        {
            Console.WriteLine($"Id: {reader["Id"]}, Name: {reader["Name"]}, Capital: {reader["Capital"]}, Region: {reader["Region"]}, Subregion: {reader["Subregion"]}");
        }
    }
}

// Query processing and optimization
string optimizedQuery = "SELECT * FROM Countries WHERE Region = 'Asia' AND Subregion = 'Southern Asia' LIMIT 5;";
using (SqliteCommand optimizedCommand = new SqliteCommand(optimizedQuery, connection))
{
    using (SqliteDataReader reader = optimizedCommand.ExecuteReader())
    {
        Console.WriteLine("Optimized Query Results:");
        while (reader.Read())
        {
            Console.WriteLine($"Id: {reader["Id"]}, Name: {reader["Name"]}, Capital: {reader["Capital"]}, Region: {reader["Region"]}, Subregion: {reader["Subregion"]}");
        }
    }
}



// Concurrency control
// SQLite supports built-in concurrency control through transactions





Query Results:
Id: 1, Name: Afghanistan, Capital: Kabul, Region: Asia, Subregion: Southern Asia
Id: 12, Name: Armenia, Capital: Yerevan, Region: Asia, Subregion: Western Asia
Id: 16, Name: Azerbaijan, Capital: Baku, Region: Asia, Subregion: Western Asia
Id: 18, Name: Bahrain, Capital: Manama, Region: Asia, Subregion: Western Asia
Id: 19, Name: Bangladesh, Capital: Dhaka, Region: Asia, Subregion: Southern Asia
Optimized Query Results:
Id: 1, Name: Afghanistan, Capital: Kabul, Region: Asia, Subregion: Southern Asia
Id: 19, Name: Bangladesh, Capital: Dhaka, Region: Asia, Subregion: Southern Asia
Id: 26, Name: Bhutan, Capital: Thimphu, Region: Asia, Subregion: Southern Asia
Id: 101, Name: India, Capital: New Delhi, Region: Asia, Subregion: Southern Asia
Id: 103, Name: Iran, Capital: Tehran, Region: Asia, Subregion: Southern Asia


In [50]:
// Transaction processing
try
{
    using (SqliteTransaction transaction1 = connection.BeginTransaction())
    {
        // Perform some updates within a transaction
        string updateQuery1 = "UPDATE Countries SET Capital = 'New Capital' WHERE Region = 'Asia';";
        using (SqliteCommand updateCommand1 = new SqliteCommand(updateQuery1, connection, transaction1))
        {
            updateCommand1.ExecuteNonQuery();
        }

        // Simulate an error to roll back the transaction
        // throw new Exception("Simulated error");

        // Commit the transaction if no error occurred
        transaction1.Commit();
    }
}
catch (Exception ex)
{
    Console.WriteLine($"Transaction rolled back: {ex.Message}");
}

// Close the main connection
connection.Close();

In [51]:
// Open two connections for simulating concurrent access
SqliteConnection connection1 = new SqliteConnection(connectionString);
SqliteConnection connection2 = new SqliteConnection(connectionString);

connection1.Open();
connection2.Open();

try
{
    using (SqliteTransaction transaction1 = connection1.BeginTransaction())
    {
        // Connection 1 updates data
        string updateQuery1 = "UPDATE Countries SET Capital = 'Updated Capital 1' WHERE Region = 'Asia';";
        using (SqliteCommand updateCommand1 = new SqliteCommand(updateQuery1, connection1, transaction1))
        {
            updateCommand1.ExecuteNonQuery();
        }

        // Simulate a delay to allow connection2 to attempt an update
        System.Threading.Thread.Sleep(5000);

        using (SqliteTransaction transaction2 = connection2.BeginTransaction())
        {
            // Connection 2 attempts to update the same data
            string updateQuery2 = "UPDATE Countries SET Capital = 'Updated Capital 2' WHERE Region = 'Asia';";
            using (SqliteCommand updateCommand2 = new SqliteCommand(updateQuery2, connection2, transaction2))
            {
                updateCommand2.ExecuteNonQuery();
            }

            // Commit transaction 2
            transaction2.Commit();
        }

        // Commit transaction 1
        transaction1.Commit();
    }
}
catch (Exception ex)
{
    Console.WriteLine($"Concurrency control exception: {ex.Message}");
}
finally
{
    // Close connections
    connection1.Close();
    connection2.Close();
}

Concurrency control exception: SQLite Error 5: 'database is locked'.
